## Capstone Project

In [1]:
## Installing and Importing Python Libraries and Dependencies

In [2]:
!pip install geocoder
!pip install folium

In [3]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


ModuleNotFoundError: No module named 'geopy'

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')

In [6]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [7]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [8]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M1B\n,Scarborough\n,Malvern / Rouge
2,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
3,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
4,M1G\n,Scarborough\n,Woburn


In [9]:
df.describe()

,Postalcode,Borough,Neighborhood
count,180,180,180
unique,180,11,99
top,M3L\n,Not assigned\n,
freq,1,77,77


In [10]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [11]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M1B\n,Scarborough\n,Malvern / Rouge
2,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
3,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
4,M1G\n,Scarborough\n,Woburn


In [12]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [13]:
df_2.describe()

,Postalcode,Borough,Neighborhood
count,180,180,180
unique,180,11,99
top,M3L\n,Not assigned\n,
freq,1,77,77


In [14]:
print(df_2.shape)
df_2.head()

(180, 3)


,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M1B\n,Scarborough\n,Malvern / Rouge
2,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
3,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
4,M1G\n,Scarborough\n,Woburn


In [15]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.70941386000004, -79.36309957799995]

In [16]:
# Retrieving Postal Code Co-ordinates
postal_codes = df_2['Postalcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [17]:
# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_2['Latitude'] = df_coords['Latitude']
df_2['Longitude'] = df_coords['Longitude']


In [18]:
df_2[df_2.Postalcode == 'M5G']

,Postalcode,Borough,Neighborhood,Latitude,Longitude


In [19]:
df_2.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,,43.648690,-79.385440
1,M1B\n,Scarborough\n,Malvern / Rouge,43.808626,-79.189913
2,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368
3,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill,43.765806,-79.185284
4,M1G\n,Scarborough\n,Woburn,43.771545,-79.218135
5,M1H\n,Scarborough\n,Cedarbrae,43.768791,-79.238813
6,M1J\n,Scarborough\n,Scarborough Village,43.744203,-79.228725
7,M1K\n,Scarborough\n,Kennedy Park / Ionview / East Birchmount Park,43.726881,-79.265694
8,M1L\n,Scarborough\n,Golden Mile / Clairlea / Oakridge,43.713340,-79.284942
9,M1M\n,Scarborough\n,Cliffside / Cliffcrest / Scarborough Village West,43.723538,-79.228353


In [20]:
address = 'Scarborough,Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geograpical Co-ordinate of Seattle,Washington are {}, {}.'.format(latitude_x, longitude_y))

NameError: name 'Nominatim' is not defined

## 3. Map of Scarborough

In [ ]:

for lat, lng, nei in zip(df_2['Latitude'], df_2['Longitude'], df_2['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

In [ ]:
address = 'Scarborough,Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_n1 = location.latitude
longitude_n1 = location.longitude
print('The Geograpical Co-ordinate of Neighborhood_1 are {}, {}.'.format(latitude_x, longitude_y))

In [ ]:
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # my Foursquare ID
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

In [ ]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_n1, 
   longitude_n1, 
    radius, 
   LIMIT)
results = requests.get(url).json()

In [ ]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## 4. Nearby Venues/Locations

In [ ]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

	venue.name	venue.categories	venue.location.lat	venue.location.lng
0	Disney Store	[{'id': '4bf58dd8d48988d1f3941735', 'name': 'T...	43.775537	-79.256833
1	American Eagle Outfitters	[{'id': '4bf58dd8d48988d103951735', 'name': 'C...	43.775908	-79.258352
2	SEPHORA	[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...	43.775017	-79.258109
3	Canyon Creek Chophouse	[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...	43.776959	-79.261694
4	Coliseum Scarborough Cinemas	[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...	43.775995	-79.255649

## 5. Categories of Nearby Venues/Locations

In [ ]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(5)

In [ ]:
# Top 10 Categories
a=pd.Series(nearby_venues.categories)
a.value_counts()[:10]

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Scarborough_venues = getNearbyVenues(names=df_2['Neighborhood'],
                                   latitudes=df_2['Latitude'],
                                   longitudes=df_2['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Bedford Park, Lawrence Manor East
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Lawrence Heights, Lawrence Manor
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
Downsview, North Park, Upwood Park
Del Ray, Keelesdale, Mount Dennis, Silverthorn
The Junction North, Runnymede
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
Queen's Park
Canada Post Gateway Processing Centre
Business Reply Mail Processing Centre 969 Eastern
Humber Bay Shores, Mimico South, New Toronto
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor
Islington Avenue
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Humber Summit
Emery, Humberlea
Weston
Westmount
Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown
Northwest

In [ ]:
print('There are {} Uniques Categories.'.format(len(Scarborough_venues['Venue Category'].unique())))
Scarborough_venues.groupby('Neighborhood').count().head()

In [ ]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_onehot.head(5)

In [ ]:
num_top_venues = 5
for hood in Scarborough_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp =Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide, King, Richmond ----
          venue  freq 
0   Coffee Shop  0.09 
1          Café  0.07 
2         Hotel  0.06 
3  Burger Joint  0.03 
4           Bar  0.03 


---- Agincourt ----
                  venue  freq
0         Shopping Mall  0.07
1    Chinese Restaurant  0.07
2  Hong Kong Restaurant  0.03
3   Filipino Restaurant  0.03
4   Shanghai Restaurant  0.03


---- Agincourt North, L'Amoreaux East, Milliken, Steeles East ----
              venue  freq
0          Pharmacy  0.50
1    Sandwich Place  0.25
2  Sushi Restaurant  0.25
3       Zoo Exhibit  0.00
4     Movie Theater  0.00


---- Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown ----
                 venue  freq
0        Grocery Store  0.17
1       Hardware Store  0.08
2  Fried Chicken Joint  0.08
3       Sandwich Place  0.08
4       Discount Store  0.08


---- Alderwood, Long Branch ----
               venue  freq
0  Convenience Store  0.11
1        Coffee Shop  0.11
2     Sandwich Place  0.11
3       Dance Studio  0.11
4                Pub  0.11


---- Bathurst Manor, Downsview North, Wilson Heights ----
                       venue  freq
0                Pizza Place  0.18
1                Coffee Shop  0.09
2   Mediterranean Restaurant  0.09
3              Deli / Bodega  0.09
4  Middle Eastern Restaurant  0.09


---- Bayview Village ----
              venue  freq
0  Asian Restaurant  0.25
1       Flower Shop  0.25
2              Park  0.25
3             Trail  0.25
4       Zoo Exhibit  0.00


---- Bedford Park, Lawrence Manor East ----
                venue  freq
0    Sushi Restaurant  0.07
1         Coffee Shop  0.07
2  Italian Restaurant  0.07
3                Café  0.04
4           Juice Bar  0.04


---- Berczy Park ----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.05
2         Café  0.05
3        Hotel  0.05
4     Beer Bar  0.04


---- Birch Cliff, Cliffside West ----
                   venue  freq
0                   Park  0.22
1         Baseball Field  0.11
2           Skating Rink  0.11
3  General Entertainment  0.11
4                    Gym  0.11


---- Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe ----
               venue  freq
0     Baseball Field  0.09
1  Fish & Chips Shop  0.09
2     Sandwich Place  0.09
3      Shopping Mall  0.09
4      Grocery Store  0.09


---- Brockton, Exhibition Place, Parkdale Village ----
                    venue  freq
0             Coffee Shop  0.08
1                    Café  0.07
2                     Bar  0.06
3              Restaurant  0.04
4  Furniture / Home Store  0.04


---- Business Reply Mail Processing Centre 969 Eastern ----
              venue  freq
0       Coffee Shop  0.10
1              Café  0.06
2             Hotel  0.04
3               Bar  0.04
4  Sushi Restaurant  0.03


---- CFB Toronto, Downsview East ----
          venue  freq
0  Soccer Field   0.2
1    Food Court   0.2
2       Airport   0.2
3          Park   0.2
4   Coffee Shop   0.2


---- CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara ----
                venue  freq
0         Coffee Shop  0.10
1          Restaurant  0.05
2                Café  0.05
3  Italian Restaurant  0.05
4                 Gym  0.04


---- Cabbagetown, St. James Town ----
                 venue  freq
0           Restaurant  0.06
1          Coffee Shop  0.06
2          Pizza Place  0.05
3  Japanese Restaurant  0.03
4               Bakery  0.03


---- Caledonia-Fairbanks ----
                 venue  freq
0                 Park  0.17
1  Sporting Goods Shop  0.08
2             Bus Stop  0.08
3                 Café  0.08
4               Market  0.08


---- Canada Post Gateway Processing Centre ----
              venue  freq
0       Coffee Shop  0.10
1              Café  0.06
2             Hotel  0.04
3               Bar  0.04
4  Sushi Restaurant  0.03


---- Cedarbrae ----
                  venue  freq
0  Caribbean Restaurant  0.12
1           Flower Shop  0.12
2     Indian Restaurant  0.12
3                  Bank  0.12
4                Bakery  0.12


---- Central Bay Street ----
                venue  freq
0         Coffee Shop  0.06
1      Clothing Store  0.04
2      Cosmetics Shop  0.04
3      Ice Cream Shop  0.03
4  Italian Restaurant  0.03


---- Chinatown, Grange Park, Kensington Market ----
                           venue  freq
0                           Café  0.06
1             Chinese Restaurant  0.05
2                            Bar  0.05
3          Vietnamese Restaurant  0.04
4  Vegetarian / Vegan Restaurant  0.04


---- Christie ----
               venue  freq
0  Korean Restaurant  0.18
1      Grocery Store  0.10
2              Diner  0.04
3  Indian Restaurant  0.04
4               Café  0.04


---- Church and Wellesley ----
                 venue  freq
0          Coffee Shop  0.09
1  Japanese Restaurant  0.04
2                 Café  0.03
3     Sushi Restaurant  0.03
4           Restaurant  0.03


---- Clairlea, Golden Mile, Oakridge ----
          venue  freq
0  Intersection  0.17
1      Bus Line  0.11
2   Coffee Shop  0.11
3         Diner  0.11
4        Bakery  0.11


---- Clarks Corners, Sullivan, Tam O'Shanter ----
                   venue  freq
0   Fast Food Restaurant  0.10
1            Pizza Place  0.10
2  Vietnamese Restaurant  0.07
3            Golf Course  0.03
4                 Market  0.03


---- Cliffcrest, Cliffside, Scarborough Village West ----
                  venue  freq
0  Fast Food Restaurant  0.17
1                  Park  0.08
2           Flower Shop  0.08
3          Liquor Store  0.08
4           Pizza Place  0.08


---- Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park ----
                venue  freq
0         Pizza Place  0.25
1   Convenience Store  0.12
2  Mexican Restaurant  0.12
3                Bank  0.12
4                 Gym  0.12


---- Commerce Court, Victoria Hotel ----
         venue  freq
0  Coffee Shop  0.10
1        Hotel  0.06
2         Café  0.05
3   Restaurant  0.04
4    Gastropub  0.04


---- Davisville ----
            venue  freq
0    Dessert Shop  0.10
1     Pizza Place  0.10
2     Coffee Shop  0.07
3  Sandwich Place  0.07
4             Gym  0.05


---- Davisville North ----
            venue  freq
0             Gym  0.09
1     Yoga Studio  0.04
2        Pharmacy  0.04
3  Breakfast Spot  0.04
4         Brewery  0.04


---- Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West ----
              venue  freq
0               Pub  0.10
1       Coffee Shop  0.10
2      Skating Rink  0.10
3  Sushi Restaurant  0.10
4       Yoga Studio  0.05


---- Del Ray, Keelesdale, Mount Dennis, Silverthorn ----
                        venue  freq
0  Construction & Landscaping   0.2
1                  Playground   0.2
2            Storage Facility   0.2
3                 Coffee Shop   0.2
4        Fast Food Restaurant   0.2


In [22]:
kclusters = 10

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude_x, longitude_y], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map

markers_colors = []
for lat, lon, nei , cluster in zip(Scarborough_merged['Latitude'], 
                                   Scarborough_merged['Longitude'], 
                                   Scarborough_merged['Neighborhood'], 
                                   Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
Scarborough_Avg_HousingPrice.set_index('Neighborhood',inplace=True,drop=True)

In [ ]:
Scarborough_Avg_HousingPrice.plot(kind='bar',figsize=(24,18),alpha=0.75)